In [ ]:
# 爬取 EPS

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep

Chromedriver_path = "/usr/local/bin/chromedriver"
EPS_file_path = "/Users/shuminyang/Desktop/ccProject/EPS"
EPS_URL = "https://goodinfo.tw/StockInfo/StockList.asp?SHEET=%E5%B9%B4%E7%8D%B2%E5%88%A9%E8%83%BD%E5%8A%9B&MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E5%B9%B4%E5%BA%A6EPS%E6%9C%80%E9%AB%98"
account = "mintesteronly"
password = "***************"

# 設定 Chrome 以隱身模式開啟（清除 cookie 資料）
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

#在 webdriver options 指定檔案存取路徑
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': EPS_file_path}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(executable_path=Chromedriver_path, chrome_options=chrome_options)
driver.get(EPS_URL)
driver.find_element_by_id("linkLoginGoogle").click() #Login
driver.find_element_by_id("identifierId").send_keys(account)
driver.find_element_by_id("identifierId").send_keys(Keys.RETURN)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "password"))) #wait
driver.find_element_by_name("password").send_keys(password)
driver.find_element_by_name("password").send_keys(Keys.RETURN)
sleep(3) #讓網頁 loading 完成後再選取下拉選單

# 連續下載近十年資料
for year in range(1,11):
    Select(driver.find_element_by_id('selRPT_TIME')).select_by_index(year)
    sleep(3) #loading
    Select(driver.find_element_by_id('selRANK')).select_by_visible_text(u"全部顯示")
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "row300"))) #wait
    driver.find_element_by_xpath("//input[@value='匯出CSV']").click()
    sleep(3) #等待檔案下載完成再關閉網頁
    
driver.quit()

In [ ]:
# 取 殖利率＆現金股利＆股票股利

file_path = "/Users/shuminyang/Desktop/ccProject/Dividend yield"
URL = "https://goodinfo.tw/StockInfo/StockList.asp?SHEET=%E8%82%A1%E5%88%A9%E6%94%BF%E7%AD%96&MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E7%8F%BE%E9%87%91%E6%AE%96%E5%88%A9%E7%8E%87+%28%E6%9C%80%E6%96%B0%E5%B9%B4%E5%BA%A6%29%40%40%E7%8F%BE%E9%87%91%E6%AE%96%E5%88%A9%E7%8E%87%40%40%E6%9C%80%E6%96%B0%E5%B9%B4%E5%BA%A6"

#在 webdriver options 指定檔案存取路徑
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': file_path}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(executable_path=Chromedriver_path, chrome_options=chrome_options)
driver.get(URL)
driver.find_element_by_id("linkLoginGoogle").click() #Login
driver.find_element_by_id("identifierId").send_keys(account)
driver.find_element_by_id("identifierId").send_keys(Keys.RETURN)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "password"))) #wait
driver.find_element_by_name("password").send_keys(password)
driver.find_element_by_name("password").send_keys(Keys.RETURN)
sleep(5) #讓網頁 loading 完成後再選取下拉選單

# 連續下載近十年資料 (殖利率最新資料為2020年)
for year in range(1,12):
    Select(driver.find_element_by_id('selRPT_TIME')).select_by_index(year)
    sleep(3) #loading
    Select(driver.find_element_by_id('selRANK')).select_by_visible_text(u"全部顯示")
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "row300"))) #wait
    driver.find_element_by_xpath("//input[@value='匯出CSV']").click()
    sleep(3) #等待檔案下載完成再關閉網頁

driver.quit()

In [ ]:
# 依據下載時間來更改檔名

import os
#file_path = "/Users/shuminyang/Desktop/ccProject/EPS"
#file_path = "/Users/shuminyang/Desktop/ccProject/Dividend_yield"

file_path = input("輸入欲更改檔名之檔案路徑")
new_file_name = input("輸入新設定檔名，去掉「年份&csv」")

files = os.listdir(file_path) #返回檔案夾列表
files = sorted(files,key=lambda x: os.path.getmtime(os.path.join(file_path, x))) #依據更新時間舊到新排序檔案
print(files) #印出讀取到的檔名稱，用來確認自己是不是真的有讀到


n=0 #設定初始值 #去掉第一個檔案 .DS_Store
#print(os.getcwd()) #印出目前所在位置
for i in files[0:10]: #因為資料夾裡面的檔案都要重新更換名稱
    oldname=files[n] #指出檔案現在的路徑名稱，[n]表示第n個檔案
    newname=new_file_name+"_"+str(2019-n)+".csv" #在本案例中的命名規則為：年份+ - + 次序，最後一個.wav表示該檔案的型別
    os.chdir(file_path) #至該檔案路徑
    os.rename(oldname,newname) #重新命名檔案
    print(oldname+">>>"+newname) #印出原名與更名後的新名，可以進一步的確認每個檔案的新舊對應
    n=n+1 #當有不止一個檔案的時候，依次對每一個檔案進行上面的流程，直到更換完畢就會結束

In [ ]:
# 彙整十年資料 ！！殖利率的值裡面有 null & "0"！！

import pandas as pd
import os
import csv

column_name = input("輸入想要的欄位名稱, ex: EPS(元)")
dic_name = input("輸入想要的字典名稱, ex: EPS_dic")
open_file_name = input("輸入要開啟的檔案名稱（去掉年份和csv）, ex: EPS")
save_file_name = input("輸入想要儲存的檔名, ex: EPS.csv")
path = input("/Users/shuminyang/Desktop/ccProject/EPS or Dividend")
dic_name = {}
No_data = {}
        
#將10年的資料存到 dic_name
os.chdir(path) #改變當前工作目錄到指定路徑
for year in range(2019,2009,-1):
    temp_lst = []
    file = open_file_name+"_"+str(year)+".csv"
    with open(file, newline='') as csvfile: #以 csvfile 模式開啟 CSV 檔
        rows = csv.DictReader(csvfile) #DictReader 會把每一列資料變成一個字典，且字典的key為第一列之每個欄位值
        for row in rows:
            
            #有些代號會出現標點符號，需過濾掉
            if "=" in row["代號"]:
                stock_symbol = row["代號"][2:-1:1]
            else:
                stock_symbol = row["代號"]
            
            if stock_symbol in dic_name: #如果該列的公司名稱已存在在 dic_name
                dic_name[stock_symbol].append(row[column_name]) #則加入該年度的value 至該公司的list
            else:
                dic_name[stock_symbol] = [row["名稱"]]
                dic_name[stock_symbol].append(row[column_name])
                No_data[year] = temp_lst.append(row["名稱"])
                
print(No_data) #檢查是否每年公司數目皆相同，若 No_data 字典為空則是


#設定显示所有列&行
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#將dic存成Dataframe格式
column1 = ["代號","名稱"]
column2 = list(range(2019,2009,-1))
column = column1+column2
#將 key 值當作 Index，再重新設定 Index
df = pd.DataFrame.from_dict(dic_name, orient="index").reset_index()
df.columns = column
df.to_csv(save_file_name, index=False) #將彙整的檔案匯出成 csv 檔 #index=False不要另外新增index欄位

In [ ]:
#計算各指標之平均值

import pandas as pd
import numpy as np
import os

#axis約簡的軸。DataFrame的欄用0，列用1(預設為欄)
#設定显示所有列&行
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

EPS = pd.read_csv("/Users/shuminyang/Desktop/ccProject/EPS/EPS.csv", index_col="代號")
DividendYield = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Dividend/DividendYield.csv", index_col="代號")
CashDividend = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Dividend/CashDividend.csv", index_col="代號")
StockDividend = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Dividend/StockDividend.csv", index_col="")
EPS = pd.DataFrame(EPS)
DividendYield = pd.DataFrame(DividendYield)
CashDividend = pd.DataFrame(CashDividend)
StockDividend = pd.DataFrame(StockDividend)

#抓取所有公司名稱存到 cols list (EPS不會用到)
cols_DividendYield = list(DividendYield.columns)
cols_CashDividend = list(CashDividend.columns)
cols_StockDividend = list(StockDividend.columns)

#一次性將nan取代所有值為0的資料 (EPS不用去掉0的值)
DividendYield = DividendYield.replace({0:np.nan})
CashDividend = CashDividend.replace({0:np.nan})
StockDividend = StockDividend.replace({0:np.nan})

#殖利率平均：找出連續五年以上的資料才納入計算
DividendYield_overFiveYear_dic = {}
for col in cols_DividendYield: #檢查該公司
    count = 0 #count初值
    DividendYield_overFiveYear_dic[col] = [] #每間公司 value 的初值為空 list
    for year in DividendYield.index: #檢查該公司之每個年度是否有資料
        year_data = DividendYield.loc[year,col] #取該公司該年度的值
        #remain_years_and_ = len(DividendYield_overFiveYear_dic[col])+(year-2009)
    
        #如果值為空且累積年份小於5，則歸零count重新計算，且把字典中已紀錄的年份清空
        if np.isnan(year_data) == True and count < 5: 
            count=0
            DividendYield_overFiveYear_dic[col] = []
            #如果剩餘的年度不到五年，則該公司不可能會有累積五年以上之資料，提早結束迴圈
            if (year-2009) < 5:
                del DividendYield_overFiveYear_dic[col]
                break
                
        #反之，如果值為空但累積年份已大於5，則提早結束迴圈
        elif np.isnan(year_data) == True and count >= 5:
            break
        #如果有值，且（已累積年份+剩餘年份）小於5，則該公司不可能會有累積五年以上之資料，提早結束迴圈
        elif np.isnan(year_data) != True and count+(year-2009) < 5:
            del DividendYield_overFiveYear_dic[col]
            break
        #反之，如果有值，且（已累積年份+剩餘年份）大於5，則 count值+1，且把該年年份新增到字典
        elif np.isnan(year_data) != True and count+(year-2009) >= 5:
            DividendYield_overFiveYear_dic[col].append(year)
            count+=1
            
            
#現金股利：找出連續五年以上的資料才納入計算
CashDividend_overFiveYear_dic = {}
for col in cols_CashDividend: #檢查該公司
    count = 0 #count初值
    CashDividend_overFiveYear_dic[col] = [] #每間公司 value 的初值為空 list
    for year in CashDividend.index: #檢查該公司之每個年度是否有資料
        year_data = CashDividend.loc[year,col] #取該公司該年度的值
    
        #如果值為空且累積年份小於5，則歸零count重新計算，且把字典中已紀錄的年份清空
        if np.isnan(year_data) == True and count < 5: 
            count=0
            CashDividend_overFiveYear_dic[col] = []
            #如果剩餘的年度不到五年，則該公司不可能會有累積五年以上之資料，提早結束迴圈
            if (year-2009) < 5:
                del CashDividend_overFiveYear_dic[col]
                break
                
        #反之，如果值為空但累積年份已大於5，則提早結束迴圈
        elif np.isnan(year_data) == True and count >= 5:
            break
        #如果有值，且（已累積年份+剩餘年份）小於5，則該公司不可能會有累積五年以上之資料，提早結束迴圈
        elif np.isnan(year_data) != True and count+(year-2009) < 5:
            del CashDividend_overFiveYear_dic[col]
            break
        #反之，如果有值，且（已累積年份+剩餘年份）大於5，則 count值+1，且把該年年份新增到字典
        elif np.isnan(year_data) != True and count+(year-2009) >= 5:
            CashDividend_overFiveYear_dic[col].append(year)
            count+=1

            
#股票股利：找出連續五年以上的資料才納入計算
StockDividend_overFiveYear_dic = {}
for col in cols_StockDividend: #檢查該公司
    count = 0 #count初值
    StockDividend_overFiveYear_dic[col] = [] #每間公司 value 的初值為空 list
    for year in StockDividend.index: #檢查該公司之每個年度是否有資料
        year_data = StockDividend.loc[year,col] #取該公司該年度的值
        #remain_years_and_ = len(DividendYield_overFiveYear_dic[col])+(year-2009)
    
        #如果值為空且累積年份小於5，則歸零count重新計算，且把字典中已紀錄的年份清空
        if np.isnan(year_data) == True and count < 5: 
            count=0
            StockDividend_overFiveYear_dic[col] = []
            #如果剩餘的年度不到五年，則該公司不可能會有累積五年以上之資料，提早結束迴圈
            if (year-2009) < 5:
                del StockDividend_overFiveYear_dic[col]
                break
                
        #反之，如果值為空但累積年份已大於5，則提早結束迴圈
        elif np.isnan(year_data) == True and count >= 5:
            break
        #如果有值，且（已累積年份+剩餘年份）小於5，則該公司不可能會有累積五年以上之資料，提早結束迴圈
        elif np.isnan(year_data) != True and count+(year-2009) < 5:
            del StockDividend_overFiveYear_dic[col]
            break
        #反之，如果有值，且（已累積年份+剩餘年份）大於5，則 count值+1，且把該年年份新增到字典
        elif np.isnan(year_data) != True and count+(year-2009) >= 5:
            StockDividend_overFiveYear_dic[col].append(year)
            count+=1


#求EPS平均，並存成字典
EPS_average_dic = EPS.mean().to_dict()
            
#求殖利率平均值，並轉成字典
DividendYield_average_dic = {}
for key,value in DividendYield_overFiveYear_dic.items():
    DividendYield_average_dic[key] = DividendYield.loc[value, key].mean()

#求現金股利平均值，並存成字典
CashDividend_average_dic = {}
for key,value in CashDividend_overFiveYear_dic.items():
    CashDividend_average_dic[key] = CashDividend.loc[value, key].mean()
    
#求股票股利平均值，並存成字典
StockDividend_average_dic = {}
for key,value in StockDividend_overFiveYear_dic.items():
    StockDividend_average_dic[key] = StockDividend.loc[value, key].mean()

#將所有指標之平均值統整到 DataFrame
average_df = pd.DataFrame([EPS_average_dic, DividendYield_average_dic, CashDividend_average_dic, StockDividend_average_dic])

#插入指標名稱在 DataFrame 第一個欄位，之後當 index 用
average_df.insert(0,"index",["EPS", "DividendYield", "CashDividend", "StockDividend"])
#將 DataFrame 存成 csv 檔
average_df.to_csv("/Users/shuminyang/Desktop/ccProject/Average.csv", index=False)

"""
印出EPS 值為 null 的資料，但只能用欄位當索引，不能用index，所以要轉置，code如下:
average_df.T[average_df.T["EPS"].isnull()]
"""

In [ ]:
import pandas as pd

Average = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average.csv", index_col="代號")
Average = pd.DataFrame(Average)
Average_del_NanEps = Average.copy() # 複製一個 Dataframe

for col in Average_del_NanEps.columns:
    EPS_data = Average_del_NanEps.loc["EPS",col]
    if pd.isnull(EPS_data) == True:
        Average_del_NanEps.drop(col,axis=1,inplace=True)

# 存檔：已刪除 etf, 特別股版本
#Average_del_NanEps.reset_index().to_csv("/Users/shuminyang/Desktop/ccProject/Average_final.csv", index=False)        
#Average_del_NanEps.reset_index().to_html("/Users/shuminyang/Desktop/ccProject/Average_final.html", index=False)        

# 為了之後要排名，將Nan的值用0代替
Average_del_NanEps = Average_del_NanEps.fillna(0)
for row in Average_del_NanEps.index[1:]:
    Average_del_NanEps.loc[row] = pd.to_numeric(Average_del_NanEps.loc[row], errors='coerce')

# 排名各指標(row)，若相同值則名次給一樣
rank = Average_del_NanEps.loc["EPS":].rank(axis=1,ascending=False,method="min")
# 把原檔的指標值刪除，只留股票代號和名稱，以利後續和排名的 dataframe 合併
name = Average_del_NanEps.drop(["EPS","DividendYield","CashDividend","StockDividend"])

# 合併股票名稱和名次兩個 dataframe
Average_rank = pd.concat([name, rank]).reset_index()
# 存檔
#Average_rank.to_csv("/Users/shuminyang/Desktop/ccProject/Average_rank.csv", index=False)
#Average_rank.to_html("/Users/shuminyang/Desktop/ccProject/Average_rank.html", index=False)
Average_rank

In [ ]:
# 忘記將先前的檔案轉向了

Average_final = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average_final.csv", index_col="代號")
Average_final = pd.DataFrame(Average_final).reset_index()
#Average_final.rename(columns={'index': '代號'}, inplace=True)
#Average_final.to_html("/Users/shuminyang/Desktop/ccProject/Average_final.html", index=False)
#Average_final.to_csv("/Users/shuminyang/Desktop/ccProject/Average_final.csv", index=False)


Average_rank = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average_rank.csv", index_col="代號")
Average_rank = pd.DataFrame(Average_rank).reset_index()
#Average_rank.rename(columns={'index': '代號'}, inplace=True)
#Average_rank.to_html("/Users/shuminyang/Desktop/ccProject/Average_rank.html", index=False)
#Average_rank.to_csv("/Users/shuminyang/Desktop/ccProject/Average_rank.csv", index=False)

Average = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average.csv", index_col="代號")
Average = pd.DataFrame(Average).reset_index()
#Average.rename(columns={'index': '代號'}, inplace=True)
#Average.to_html("/Users/shuminyang/Desktop/ccProject/Average.html", index=False)
#Average.to_csv("/Users/shuminyang/Desktop/ccProject/Average.csv", index=False)

In [ ]:
import pandas as pd
Average_rank_beta = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average_rank_beta.csv", index_col="index")
Average_rank_beta = pd.DataFrame(Average_rank_beta).T
Average_rank = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Average_rank.csv", index_col="代號")
Average_rank = pd.DataFrame(Average_rank)

# 轉成字典並合併為一個字典 Average_rank_merge
# Dataframe 轉成字典時會把 index(公司代號) 設成 int 型態
Average_rank = Average_rank.to_dict()
Average_rank_beta = Average_rank_beta.to_dict()
Average_name_beta = Average_rank_beta.pop('名稱')
Average_rank_merge = {**Average_rank, **Average_rank_beta} #合併字典，若key相同 後者覆蓋前者（我們不會遇到）

# 將公司代號改成字串型態，不然對不起來
for K,V in Average_rank_merge.items():
    temp_dic = dict()
    for k,v in Average_rank_merge[K].items():
        temp_dic[str(k)] = v
    Average_rank_merge[K] = temp_dic

# 將字典轉為 dataframe
Average_rank_merge = pd.DataFrame(Average_rank_merge)

# 把 EPS 為空的列刪除
for row in Average_rank_merge.index:
    EPS_data = Average_rank_merge.loc[row,"EPS"]
    if pd.isnull(EPS_data) == True:
        Average_rank_merge.drop(row,axis=0,inplace=True)


# 抓欄位資料
cols = Average_rank_merge.columns.drop('名稱')
Average_rank_merge[cols] = Average_rank_merge[cols].apply(pd.to_numeric, errors='coerce')

#重新排序第四到第八欄 cols[4:8]
Average_rank_merge[cols[4:8]] = Average_rank_merge[cols[4:8]].rank(ascending=True,method="min")

# 輸出Dataframe
Average_rank_merge = Average_rank_merge.reset_index()
#reset_index 完後index那一欄的欄位名稱會被強制改叫做「index」，需手動改回
Average_rank_merge.rename(columns={'index': '代號'}, inplace=True)
Average_rank_merge.to_csv("/Users/shuminyang/Desktop/ccProject/Rank.csv", index=False)

In [ ]:
# 把被刪除的 nan 資料印出來

num = list()
name = list()
for number in Average_rank_merge.index:
    EPS = Average_rank_merge.loc[number,"EPS"]
    if pd.isnull(EPS) == True:
        num.append(number)
        name.append(Average_rank_merge.loc[number, "名稱"])
d = {'代號1':num[0:143],'名稱1':name[0:143],'代號2':num[143:287],'名稱2':name[143:287]}
pd.DataFrame(d)
pd.DataFrame(d).to_csv("/Users/shuminyang/Desktop/ccProject/nan.csv")

In [13]:
# 刪掉 1902-台紙 公司

import pandas as pd
rank = pd.read_csv("/Users/shuminyang/Desktop/ccProject/Rank.csv", index_col="代號")
rank = pd.DataFrame(rank)
rank = rank.drop(index=1902)

rank = rank.reset_index()
#reset_index 完後index那一欄的欄位名稱會被強制改叫做「index」，需手動改回
rank.rename(columns={'index': '代號'}, inplace=True)
rank.to_csv("/Users/shuminyang/Desktop/ccProject/Rank2.csv", index=False)